Kindle Sentiment analysis

In [144]:
import pandas as pd

In [145]:
df=pd.read_csv(r"C:\Users\Admin\Desktop\Madhu\udemyproject\NLP practice\kindle\archive (8)\all_kindle_review .csv")

In [146]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [147]:
df=df[['reviewText','rating']]

In [148]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [149]:
df.info

<bound method DataFrame.info of                                               reviewText  rating
0      Jace Rankin may be short, but he's nothing to ...       3
1      Great short read.  I didn't want to put it dow...       5
2      I'll start by saying this is the first of four...       3
3      Aggie is Angela Lansbury who carries pocketboo...       3
4      I did not expect this type of book to be in li...       4
...                                                  ...     ...
11995  Valentine cupid is a vampire- Jena and Ian ano...       4
11996  I have read all seven books in this series. Ap...       5
11997  This book really just wasn't my cuppa.  The si...       3
11998  tried to use it to charge my kindle, it didn't...       1
11999  Taking Instruction is a look into the often hi...       3

[12000 rows x 2 columns]>

In [150]:
df.shape

(12000, 2)

In [151]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [152]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [153]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

preprocess and cleaning

In [154]:
#positive review is 1 and negative is 0
df['rating']=df['rating'].apply(lambda x: 0 if x<3 else 1)

In [155]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [156]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [157]:
df['rating'].unique()

array([1, 0], dtype=int64)

1. Lower all cases

In [158]:
df['reviewText']=df['reviewText'].str.lower()

2. Remove special characters

In [159]:
!pip install lxml

In [160]:
import lxml
print(lxml.__version__)


5.4.0


In [161]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')
import re
from bs4 import BeautifulSoup


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [162]:
df['reviewText']=df['reviewText'].apply(lambda x: re.sub('[^a-z A-Z0-9]+','',x))
df['reviewText']=df['reviewText'].apply(lambda x: " ".join([y for y in x.split() if y not in 
                                                            stopwords.words('english')]))
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[w_-]+)+))([w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [163]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


Lemmatization - Wordnetlemmatizer

In [164]:
from nltk.stem import WordNetLemmatizer

In [165]:
lm=WordNetLemmatizer()

In [166]:
def lem(text):
    return " ".join([lm.lemmatize(word) for word in text.split()])

In [167]:
df['reviewText']=df['reviewText'].apply(lambda x: lem(x))

In [168]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


Train Test split and model generation

In [169]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

BOW,TFIDF,Word2vec

In [170]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [171]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [172]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

GaussianNB

In [173]:
from sklearn.naive_bayes import GaussianNB
nb_bow=GaussianNB().fit(X_train_bow,y_train)
nb_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [174]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [175]:
y_pred_bow=nb_bow.predict(X_test_bow)

In [176]:
y_pred_tfidf=nb_tfidf.predict(X_test_tfidf)

In [177]:
print("BOW accuracy",accuracy_score(y_test,y_pred_bow))

BOW accuracy 0.5783333333333334


In [178]:
confusion_matrix(y_test,y_pred_bow)

array([[522, 277],
       [735, 866]], dtype=int64)

In [179]:
print("Tfidf accuracy",accuracy_score(y_test,y_pred_tfidf))

Tfidf accuracy 0.57625


In [180]:
print("classification_report for BOW",classification_report(y_test,y_pred_bow))

classification_report for BOW               precision    recall  f1-score   support

           0       0.42      0.65      0.51       799
           1       0.76      0.54      0.63      1601

    accuracy                           0.58      2400
   macro avg       0.59      0.60      0.57      2400
weighted avg       0.64      0.58      0.59      2400



In [181]:
print("classification_report for Tfidf",classification_report(y_test,y_pred_tfidf))

classification_report for Tfidf               precision    recall  f1-score   support

           0       0.41      0.63      0.50       799
           1       0.75      0.55      0.63      1601

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.64      0.58      0.59      2400



Word2vec implementation

In [182]:
from sklearn.model_selection import train_test_split
X_train_wv,X_test_wv,y_train_wv,y_test_wv=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

In [183]:
import gensim

In [184]:
from gensim.models import Word2Vec

In [185]:
df['reviewText']

0        jace rankin may short he nothing mess man haul...
1        great short read didnt want put read one sitti...
2        ill start saying first four book wasnt expecti...
3        aggie angela lansbury carry pocketbook instead...
4        expect type book library pleased find price right
                               ...                        
11995    valentine cupid vampire jena ian another vampi...
11996    read seven book series apocalypticadventure on...
11997    book really wasnt cuppa situation man capturin...
11998    tried use charge kindle didnt even register ch...
11999    taking instruction look often hidden world sex...
Name: reviewText, Length: 12000, dtype: object

In [186]:
sent_train=X_train_wv.apply(lambda x: x.split()).tolist()

In [187]:
model=gensim.models.Word2Vec(sent_train,vector_size=150)

In [188]:
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'get',
 'time',
 'author',
 'series',
 'reading',
 'much',
 'first',
 'well',
 'even',
 'didnt',
 'short',
 'know',
 'could',
 'great',
 'way',
 'make',
 'sex',
 'little',
 'dont',
 'want',
 'thing',
 'two',
 'think',
 'plot',
 'find',
 'also',
 'end',
 'romance',
 'im',
 'life',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'take',
 'lot',
 'say',
 'woman',
 'written',
 'many',
 'work',
 'kindle',
 'thought',
 'give',
 'liked',
 'year',
 'found',
 'bit',
 'going',
 'interesting',
 'writing',
 'loved',
 'come',
 'better',
 'novel',
 'got',
 'another',
 'back',
 'still',
 'feel',
 'man',
 'hot',
 'reader',
 'made',
 'enough',
 'something',
 'though',
 'people',
 'review',
 'star',
 'part',
 'page',
 'need',
 'friend',
 'cant',
 'keep',
 'bad',
 'free',
 'doesnt',
 'world',
 'wasnt',
 'new',
 'recommend',
 'together',
 'relationship',
 'enjoy',
 'felt',
 'start',
 'next',
 'word',
 'main'

In [189]:
model.corpus_count

9600

In [190]:
len(model.wv.index_to_key)

8158

In [191]:
model.epochs

5

In [192]:
model.wv.similar_by_word('good')

[('decent', 0.9023233652114868),
 ('potential', 0.89444899559021),
 ('easy', 0.8912692070007324),
 ('overall', 0.8908536434173584),
 ('ok', 0.8884605169296265),
 ('cute', 0.8877103328704834),
 ('nice', 0.887262761592865),
 ('okay', 0.8847057819366455),
 ('predictable', 0.8758370876312256),
 ('line', 0.8753783702850342)]

In [193]:
model.wv['good'].shape

(150,)

In [194]:
import numpy as np

In [195]:
def avg_word2vec(doc):
    vector=[model.wv[word] for word in doc if word in model.wv.index_to_key]
    return np.mean(vector,axis=0) if vector else np.zeros(model.vector_size)

In [196]:
from tqdm import tqdm

In [197]:
X=[]
for i in tqdm(range(len(sent_train))):
    X.append(avg_word2vec(sent_train[i]))

100%|██████████| 9600/9600 [00:09<00:00, 1003.25it/s]


In [198]:
sent_test=X_test_wv.apply(lambda x: x.split()).tolist()

In [199]:
def avg_word2vec(doc):
    vector=[model.wv[word] for word in doc if word in model.wv.index_to_key]
    return np.mean(vector,axis=0) if vector else np.zeros(model.vector_size)

X_test_wv_new=[]
for i in tqdm(range(len(sent_test))):
    X_test_wv_new.append(avg_word2vec(sent_test[i]))

100%|██████████| 2400/2400 [00:02<00:00, 999.45it/s] 


In [200]:
len(X_test_wv_new)

2400

In [201]:
X_test_wv_new_array=np.array(X_test_wv_new)

In [202]:
X_test_wv_new_array

array([[ 0.20676365, -0.26643732, -0.22324198, ..., -0.2311376 ,
         0.35348883, -0.14576113],
       [ 0.28371623, -0.5494031 , -0.3096563 , ..., -0.3276717 ,
         0.5975193 , -0.1930627 ],
       [ 0.18036748, -0.2138523 , -0.33867547, ..., -0.27379045,
         0.36532056, -0.25732622],
       ...,
       [ 0.2659359 , -0.32161182, -0.4585664 , ..., -0.28378585,
         0.441979  , -0.31837928],
       [ 0.31814513, -0.40092686, -0.2663423 , ..., -0.20204131,
         0.3442757 , -0.14699864],
       [ 0.20327987, -0.39575756, -0.29414743, ..., -0.36257437,
         0.46175563, -0.22990862]], dtype=float32)

In [210]:
len(X_test_wv_new_array)

2400

In [203]:
len(X)

9600

In [204]:
X_new=np.array(X)

In [205]:
X_new.shape

(9600, 150)

In [206]:
y_train.shape

(9600,)

In [207]:
X_test.shape

(2400,)

In [208]:
from sklearn.naive_bayes import GaussianNB
nb_w2v=GaussianNB().fit(X_new,y_train)


In [211]:
y_pred_w2v=nb_w2v.predict(X_test_wv_new_array)

In [214]:
y_pred_w2v.shape

(2400,)

In [212]:
print("accuracy_score",accuracy_score(y_test,y_pred_w2v))

accuracy_score 0.58875


In [215]:
print("classification_report",classification_report(y_test,y_pred_w2v))

classification_report               precision    recall  f1-score   support

           0       0.33      0.22      0.27       799
           1       0.67      0.77      0.71      1601

    accuracy                           0.59      2400
   macro avg       0.50      0.50      0.49      2400
weighted avg       0.55      0.59      0.56      2400



In [216]:
print("confusion_matrix",confusion_matrix(y_test,y_pred_w2v))

confusion_matrix [[ 178  621]
 [ 366 1235]]


In [217]:
from sklearn.metrics import f1_score
f1=f1_score(y_test,y_pred_w2v)

In [218]:
print(f1)

0.7144923343939832
